#### kaggle - Daily News for Stock Market Prediction
뉴스를 통해 주식시장 예측
- DJIA : Dow Jones Industrial Average

In [330]:
import pandas as pd
import numpy as np

In [331]:
folder = 'stocknews/'
files = ['Combined_News_DJIA.csv','DJIA_table.csv','RedditNews.csv']

In [332]:
combined_news = pd.read_csv(folder + files[0])
stock = pd.read_csv(folder + files[1])
reddit = pd.read_csv(folder + files[2])
rnews.shape,stock.shape, reddit.shape

((1989, 27), (1989, 7), (73608, 2))

주가 데이터(수치형) 6~7 개 컬럼 + 뉴스 데이터(TEXT > vector) 2~3만개
컬럼 / open, close, low, high, volume, 그날의 뉴스 벡터

주가 데이터 > 주가 데이터 분석 모델 
뉴스 데이터 > 뉴스 분석 모델

주가 데이터 분석 모델  +  뉴스 분석 모델  >  주식 시장 예측 

날짜가 서로 다르다. 갯수도 다르다.
일단 붙이고, 누락 데이터가 있으면 버리자

In [333]:
# 날짜 타입으로 바꾸는 함수
reddit['Date'] = pd.to_datetime(reddit['Date'])
stock['Date'] = pd.to_datetime(stock['Date'])

In [334]:
stock = stock.set_index('Date') # 한번만 실행하세요

In [335]:
reddit_agre = reddit.groupby('Date').sum() # 같은 날짜의 뉴스 붙이기

In [336]:
stock_new = pd.concat((stock, reddit_agre), axis=1) #증시와 뉴스 붙이기

In [337]:
stock_new = stock_new.dropna() # 누락 행 제거
stock_new.head(1)

,Open,High,Low,Close,Volume,Adj Close,News
Date,,,,,,,
2008-08-08,11432.089844,11759.959961,11388.040039,11734.320312,212830000.0,11734.320312,"b""Georgia 'downs two Russian warplanes' as cou..."


In [338]:
reddit_1d = stock_new['News'].values.reshape([-1]) # 1차원으로 reshape 해서 어휘사전 만들기

In [347]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = TfidfVectorizer(min_df=0.03, max_df=0.97, ngram_range = (2, 2)).fit(reddit_1d)
#vect = CountVectorizer().fit(reddit_1d)
len(vect.vocabulary_)

606

In [348]:
reddit_sparse = vect.transform(reddit_1d) # TEXT 를 vector 로 변환

In [349]:
x_stock = stock_new.drop(columns='News').values #dense
x_news = reddit_sparse.toarray()  #sparse > dense 
X = np.concatenate((x_stock, x_news), axis=1)
y = stock_new['Close'].values
X.shape, y.shape

((1989, 612), (1989,))

In [375]:
#PCA
from sklearn.decomposition import PCA
news_dense = reddit_sparse.toarray()
pca = PCA(n_components=5).fit(news_dense)
news_pca = pca.transform(news_dense)
X = np.concatenate((x_stock, news_pca), axis=1)
y = stock_new['Close'].values
X.shape, y.shape

((1989, 11), (1989,))

In [376]:
#전날 X 로 다음날 y 예측하기
X = X[:-1] #최신 데이터 제거
y = y[1:]  #가장 오래된 데이터 제거

In [377]:
train_m = int(len(X)*0.75) # 과거 데이터 75% 로 학습
X_train_no_new = X[:train_m, :6]
X_test_no_new = X[train_m:, :6]
X_train = X[:train_m]
X_test = X[train_m:]
y_train = y[:train_m]
y_test = y[train_m:]

In [382]:
#분류 모델로 fit 후 score 측정 
from sklearn.linear_model import Lasso, Ridge, SGDRegressor, BayesianRidge # 예측 Regression
from sklearn.svm import SVR
model = Lasso(max_iter=100000).fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test) # 결정계수 

(0.9969592481866569, 0.932493324227404)

In [383]:
model = Ridge(max_iter=100000).fit(X_train_no_new, y_train)
model.score(X_train_no_new, y_train), model.score(X_test_no_new, y_test) # 결정계수 

C:\Users\wtime\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.57235e-18): result may not be accurate.
  overwrite_a=True).T


(0.9969592483970705, 0.932491339811705)

In [372]:
combined_news = combined_news.set_index('Date') # 인덱스 변경

KeyError: 'Date'

In [218]:
combined_news.head(1)

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [219]:
combined_news = combined_news.dropna() # 누락 제거
combined_news.isna().sum().sum() #누락 체크

0

In [220]:
sum(combined_news.index < '2014-12-31'), sum(combined_news.index > '2014-12-31')

(1607, 378)

In [221]:
X = combined_news.drop(columns='Label')
y = combined_news['Label']
combined_news.shape, X.shape, y.shape

((1986, 26), (1986, 25), (1986,))

In [222]:
x_flat = np.reshape(X.values, [-1])
x_flat.shape

(49650,)

In [223]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [239]:
# 3% 이하/ 97 % 이상 등장하는 단어는 제외, 최대 어휘 20만 제한, N그램 2(단어 2개씩 자르기)
vect = TfidfVectorizer(min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2)).fit(x_flat)

In [229]:
#vect = CountVectorizer().fit(x_flat) #문장의 단어들을 쪼개서 어휘사전을 만듭니다
#len(vect.vocabulary_)

In [230]:
x_sparse = vect.transform(x_flat) # 자연어를 벡터로 변환

In [241]:
(vect.vocabulary_)

{'of the': 1, 'in the': 0}

In [231]:
feature_n = X.shape[1]
train_m = sum(combined_news.index <= '2014-12-31')
feature_n, train_m

(25, 1608)

In [232]:
x_sparse

<49650x2 sparse matrix of type '<class 'numpy.float64'>'
	with 5998 stored elements in Compressed Sparse Row format>

선형 모델, X 의 dim이 2이어야 합니다
> 방안 1. : 뉴스1~25 를 합친다
> 방안 2. : spares_matrix 25개를 합친다(더한다)

In [233]:
x_agr = []
for i in range(len(X)):    
    one_day_news = x_sparse[i*25 : (i+1)*25] # <25x34682> 
    new_sum = np.sum(one_day_news, axis=0)   # <34682>   spares_matrix 25개를 합친다(더한다)
    x_agr.append(new_sum)
len(x_agr)

1986

In [234]:
x_2d = np.concatenate(x_agr, axis=0)
x_2d.shape, y.shape

((1986, 2), (1986,))

In [235]:
y_train = y.values[:train_m]
y_test = y.values[train_m:]
x_train = x_2d[:train_m] # 1608 일
x_test = x_2d[train_m:]
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1608, 2), (378, 2), (1608,), (378,))

In [236]:
from sklearn.linear_model import RidgeClassifier

In [237]:
model = RidgeClassifier().fit(x_train, y_train)
model.score(x_train, y_train), model.score(x_test, y_test)

(0.5379353233830846, 0.5132275132275133)